# Text classification

### Natural Language Processing and Information Extraction,  2024WS
Lecture 2, 10/18/2024

Gábor Recski

## In this lecture

- Text classification
    - Examples of text classification
    - Naive Bayes classification (SLP 4.1, 4.2)
    - Evaluation (SLP 4.7, 4.8)
    - Example: sentiment analysis (SLP 4.3)
    - Error analysis / model debugging
- N-gram language modelling (SLP 3.1)

[SLP Ch. 4](https://web.stanford.edu/~jurafsky/slp3/4.pdf)

[SLP Ch. 3](https://web.stanford.edu/~jurafsky/slp3/3.pdf)




In [1]:
import csv
from collections import Counter, defaultdict
from math import exp, log

import nltk
import numpy as np
from tqdm import tqdm

## Examples of text classification

Text classification tasks are some of the simplest and most common NLP applications.

Tasks may involve classifying **documents**, **sentences** or **words**

### Document-level classification

#### Sentiment analysis

![sa](media/sa.png)

#### Spam detection

![spam](media/spam.png)

### Sentence-/utterance-level classification

#### Intent detection

![intent](media/intent.png)

([SentiOne](https://sentione.com/blog/new-state-of-the-art-intent-detection-model-from-sentione))

### Word-level classification

#### Slot-filling

![slot](media/slot.png)

([Hung-yi Lee's Slides](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/RNN%20(v2).pdf))

## Text representation

### Bag Of Words

The most basic representation of text

![bow](media/bow.png)

([SLP Ch.4](https://web.stanford.edu/~jurafsky/slp3/4.pdf))

## Naive Bayes Classification

Let's use the bag-of-words representation for learning to classify documents.
Each document in a training dataset will be represented by its words: $d_i = (x_1, x_2, \ldots, x_n)$

For a document $d$ and class $c$, maximize the likelihood that $d$ belongs to $c$ over all classes $c\in C$:

$$\underset{c\in C}{\operatorname{argmax}}P(c|d)$$

Apply [Bayes' rule](https://en.wikipedia.org/wiki/Bayes%27_theorem):

$$P(c|d) = \frac{P(d|c)P(c)}{P(d)}$$

When classifying a document, the denominator is the same for all classes and can be dropped:

$$\underset{c\in C}{\operatorname{argmax}}P(d|c)P(c)$$

If a document is represented by features $x_1, x_2, \ldots x_n$ then:

$$P(d|c) = P(x_1, x_2, \ldots, x_n|c)$$

If we make an **assumption** that the features are independent, then:

$$P(x_1, x_2, \ldots, x_n|c) = P(x_1|c)P(x_2|c)\ldots P(x_n|c)$$

If we use the bag-of-words model, the features are the words.

$$c_{\text{NB}} = \underset{c_j\in C}{\operatorname{argmax}}P(c_j)\prod_{w_i \in D}{P(w_i|c)}$$

To avoid underflow errors, we usually implement this in log-space:

$$c_{\text{NB}} = \underset{c_j\in C}{\operatorname{argmax}}\log P(c_j)+\sum_{w_i \in D}{\log P(w_i|c)}$$

And instead of multiplying very small numbers we are now adding up weights. The class with the highest score is still the most likely class.

So how do we estimate (learn) the paramteres $P(w_i|c_j)$?

The simplest way is by counting occurrences in the training data:

$$P(w_i|c_j) = \frac{\text{count}(w_i, c_j)}{\sum_{w\in V}{\text{count}(w, c_j)}}$$

where $V$ is the **vocabulary** of all words in the training data

If a word is not in the vocabulary (out-of-vocabulary, OOV), we don't want to introduce zero probabilities. The simplest solution is Laplace (add-1) **smoothing**:

$$P(w_i|c_j) = \frac{\text{count}(w_i, c_j)+1}{\sum_{w\in V}{(\text{count}(w, c_j)+1)}}$$

Another simple solution is to leave OOVs out of the equation completely

An implementation of a Naive Bayes classifier with this simplest approach will follow later in this notebook.

### Naive Bayes may be naive, but it is powerful

- very fast, with low memory requirements
- robust to irrelevant features
- works well in domains with many equally important features
- a strong, dependable baseline for text classification

### Bag-of-words may be naive, but it is powerful

- efficient: vocabularies are large but number of relevant dimensions is limited
- interpretable: lets us understand what a model learned
- robust, low-resource, does not depend on genre, domain, language-specific resources, etc.

**In natural language, most of the information comes from words!**

(The word entropy of natural language is 12-16 bits ([Kornai 2008](https://eprints.sztaki.hu/7913/1/Kornai_1762289_ny.pdf) Section 7.1), punctuation doesn't contribute much (7% for English, see [Brown 1992](https://aclanthology.org/J92-1002.pdf)), and syntax cannot contribute more than 2 bits per word ($C_n\sim 4^n$, where $C_n$ are the [Catalan numbers](https://en.wikipedia.org/wiki/Catalan_number)))

## Evaluating classifiers

### Binary classification

![eval](media/eval.png)

([SLP Ch.4](https://web.stanford.edu/~jurafsky/slp3/4.pdf))

**F-measure** is the (weighted) harmonic mean of precision and recall

$$F_\beta = \frac{(\beta^2+1)PR}{\beta^2P+R}$$

Balanced F-measure ($\beta=1$):

$$F_1 = \frac{2PR}{P+R}$$

F1 is used everywhere in **academia** (challenges, leaderboards), but precision and recall are usually **not equally important in real-world applications**!

Small differences in F-score may not mean small differences in output. You should **always look at precision and recall separately**.

### Multiple classes

![eval2](media/eval2.png)

([SLP Ch.4](https://web.stanford.edu/~jurafsky/slp3/4.pdf))

### Macro- vs. micro-averaging

![eval3](media/eval3.png)

([SLP Ch.4](https://web.stanford.edu/~jurafsky/slp3/4.pdf))

## Implementing a BOW classifier

In [2]:
class SimpleNBClassifier():
    def __init__(self):
        self.word_count = Counter()
        self.count_by_class = defaultdict(Counter)
        self.labels = set()
    
    def count_words(self, docs):
        for words, label in docs:
            self.labels.add(label)
            for word in set(words):
                self.word_count[word] += 1
                self.count_by_class[label][word] += 1
    
    def calculate_weights(self):
        self.weights = {
            word: {
                label: log((self.count_by_class[label][word] + 1) / (count + len(self.word_count)))
                for label in self.labels}
            for word, count in self.word_count.items()}

    def get_doc_weights(self, doc):
        return {
            label: sum(
                self.weights[word][label] if word in self.weights else log(1)
                for word in doc)
            for label in self.labels}
    
    def predict_label(self, doc):
        doc_weights = self.get_doc_weights(doc)
        return sorted(doc_weights.items(), key=lambda x: -x[1])[0][0]
        

**NOTE:** this implementation is not very efficient, normally you would use an ML library with efficient data structures and algorithms. E.g. here you could have used [scikit-learn's NB implementation](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

In [3]:
nb = SimpleNBClassifier()

In [4]:
test_input = [
    (("He", "is", "nice"), "POS"),
    (("He", "is", "stupid"), "NEG")
]

In [5]:
nb.count_words(test_input)

In [6]:
nb.calculate_weights()

In [7]:
nb.weights

{'He': {'POS': -1.0986122886681098, 'NEG': -1.0986122886681098},
 'nice': {'POS': -0.916290731874155, 'NEG': -1.6094379124341003},
 'is': {'POS': -1.0986122886681098, 'NEG': -1.0986122886681098},
 'stupid': {'POS': -1.6094379124341003, 'NEG': -0.916290731874155}}

In [8]:
doc1 = ('This', 'movie', 'is', 'stupid')
doc2 = ('This', 'movie', 'is', 'nice')

In [9]:
nb.get_doc_weights(doc1)

{'POS': -2.70805020110221, 'NEG': -2.0149030205422647}

In [10]:
nb.predict_label(doc1)

'NEG'

In [11]:
nb.get_doc_weights(doc2)

{'POS': -2.0149030205422647, 'NEG': -2.70805020110221}

In [12]:
nb.predict_label(doc2)

'POS'

Let's load a real dataset: [50k movie reviews from IMDB](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

In [13]:
docs = []
with open('data/imdb_dataset.csv') as csvfile:
    reader = csv.reader(csvfile)
    for text, label in tqdm(reader):
        words = nltk.word_tokenize(text)        
        docs.append((words, label))

50000it [01:15, 664.38it/s]


In [14]:
nb = SimpleNBClassifier()

In [15]:
Counter((d[1] for d in docs)).most_common()

[('positive', 25000), ('negative', 25000)]

In [16]:
nb.count_words(docs)

In [17]:
nb.calculate_weights()

In [18]:
sentences = [
    "This movie is stupid",
    "I loved this movie",
    "This is the most famous movie by Aaron Sorkin",
]

In [19]:
sample_docs = [nltk.word_tokenize(sen) for sen in sentences]
sample_docs

[['This', 'movie', 'is', 'stupid'],
 ['I', 'loved', 'this', 'movie'],
 ['This', 'is', 'the', 'most', 'famous', 'movie', 'by', 'Aaron', 'Sorkin']]

In [20]:
labels = [nb.predict_label(doc) for doc in sample_docs]
labels

['negative', 'positive', 'positive']

In [21]:
scores = [nb.get_doc_weights(doc) for doc in sample_docs]
scores

[{'negative': -12.541662089933203, 'positive': -14.242144543286088},
 {'negative': -13.287294327272875, 'positive': -12.507978373722635},
 {'negative': -42.89235870322001, 'positive': -39.82907148625904}]

In [22]:
def show_word_weights(doc, model):
    print(('{:>12}'*4).format('word', 'pos', 'neg', 'diff'))
    print('='*48)
    threshold = 1
    for word in doc:
        if word in model.weights:
            pos_weight, neg_weight = model.weights[word]['positive'], model.weights[word]['negative']
            diff = pos_weight - neg_weight
        else:
            pos_weight, neg_weight, diff = -1, -1, 0
        p_word = word if abs(diff) < threshold else f"*{word}"
        print(f'{p_word:>12}{pos_weight:>12.2f}{neg_weight:>12.2f}{diff:>12.2f}')        

In [23]:
show_word_weights(sample_docs[0], nb)

        word         pos         neg        diff
        This       -3.02       -3.02        0.00
       movie       -2.80       -2.63       -0.17
          is       -2.36       -2.38        0.02
     *stupid       -6.07       -4.52       -1.55


In [24]:
show_word_weights(sample_docs[1], nb)

        word         pos         neg        diff
           I       -2.54       -2.46       -0.08
      *loved       -4.72       -5.81        1.10
        this       -2.46       -2.39       -0.07
       movie       -2.80       -2.63       -0.17


In [25]:
show_word_weights(sample_docs[2], nb)

        word         pos         neg        diff
        This       -3.02       -3.02        0.00
          is       -2.36       -2.38        0.02
         the       -2.29       -2.29       -0.00
        most       -3.46       -3.62        0.16
      famous       -5.49       -6.05        0.56
       movie       -2.80       -2.63       -0.17
          by       -2.92       -2.97        0.05
       Aaron       -7.88       -8.44        0.57
     *Sorkin       -9.61      -11.49        1.87


### Let's train and evaluate a model

In [26]:
len(docs)

50000

In [27]:
train_docs, dev_docs, test_docs = docs[:40000], docs[40001:45000], docs[45001:]

In [28]:
nb = SimpleNBClassifier()

In [29]:
nb.count_words(train_docs)

In [30]:
nb.calculate_weights()

In [31]:
def evaluate(model, data):
    tp, fp, tn, fn = 0, 0, 0, 0
    for doc, label in data:
        pred_label = model.predict_label(doc)
        if pred_label == 'positive':
            if label == 'positive':
                tp += 1
            else:
                fp += 1
        else:
            if label == 'positive':
                fn += 1
            else:
                tn += 1
    print(f"TP: {tp}, FP: {fp}, FN: {fn}, TN: {tn}")
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2*precision*recall) / (precision + recall)
    print(f'Precision: {precision:5.2%}, Recall: {recall:5.2%}, F1: {f1:5.2%}')

In [32]:
evaluate(nb, dev_docs)

TP: 1943, FP: 219, FN: 594, TN: 2243
Precision: 89.87%, Recall: 76.59%, F1: 82.70%


### Error analysis / model debugging

Let's try to understand the model's behavior and why it makes errors.

In [33]:
sample = dev_docs[-20:]

In [34]:
evaluate(nb, sample)

TP: 4, FP: 1, FN: 3, TN: 12
Precision: 80.00%, Recall: 57.14%, F1: 66.67%


In [35]:
def get_errors(model, data):
    FPs, FNs = [], []
    for doc, gold_label in data:
        pred_label = nb.predict_label(doc)
        if pred_label == gold_label:
            continue
        if pred_label == 'positive':
            FPs.append(doc)
        else:
            FNs.append(doc)
    return FPs, FNs        

In [36]:
FPs, FNs = get_errors(nb, sample)

In [37]:
show_word_weights(FPs[0], nb)

        word         pos         neg        diff
   Hitchcock       -6.87       -7.32        0.45
          is       -2.45       -2.46        0.02
           a       -2.40       -2.39       -0.00
       great       -3.36       -4.09        0.73
    director       -4.47       -4.25       -0.22
           .       -2.38       -2.37       -0.00
  Ironically       -8.05       -8.53        0.48
           I       -2.63       -2.55       -0.08
      mostly       -5.59       -5.63        0.04
        find       -4.13       -4.28        0.15
         his       -3.07       -3.20        0.13
       films       -3.83       -4.02        0.19
           a       -2.40       -2.39       -0.00
       total       -6.45       -5.63       -0.82
      *waste       -7.28       -4.56       -2.71
          of       -2.41       -2.41       -0.00
        time       -3.36       -3.35       -0.01
          to       -2.43       -2.41       -0.02
       watch       -3.84       -3.78       -0.06
           .       -

In [38]:
show_word_weights(FNs[0], nb)

        word         pos         neg        diff
        This       -3.12       -3.11       -0.01
       movie       -2.89       -2.72       -0.17
          is       -2.45       -2.46        0.02
        just       -3.31       -3.03       -0.28
       plain       -6.64       -5.69       -0.95
       silly       -6.08       -5.34       -0.75
           .       -2.38       -2.37       -0.00
      Almost       -7.45       -7.35       -0.09
       every       -4.32       -4.32       -0.01
       scene       -4.13       -4.03       -0.10
         has       -3.11       -3.21        0.11
        some       -3.32       -3.20       -0.13
         bit       -4.39       -4.60        0.21
          of       -2.41       -2.41       -0.00
       humor       -5.20       -5.44        0.24
           :       -3.74       -3.64       -0.10
     running       -5.82       -5.39       -0.43
        gags       -6.78       -6.82        0.04
           ,       -2.40       -2.39       -0.01
   slapstick       -

Now let's use an external sample!

In [39]:
sample2 = []
with open('data/barbie_sample.tsv') as f:
    for line in f:
        review, label = line.strip().split('\t')
        doc = nltk.word_tokenize(review)
        sample2.append((doc, label))

In [40]:
evaluate(nb, sample2)

TP: 3, FP: 0, FN: 2, TN: 5
Precision: 100.00%, Recall: 60.00%, F1: 75.00%


In [41]:
FPs, FNs = get_errors(nb, sample2)

In [42]:
show_word_weights(FNs[0], nb)

        word         pos         neg        diff
           I       -2.63       -2.55       -0.08
      really       -3.51       -3.40       -0.11
     enjoyed       -4.95       -5.83        0.88
        this       -2.54       -2.47       -0.07
      little       -3.89       -3.95        0.06
       thing       -4.41       -3.91       -0.50
           !       -3.31       -3.27       -0.04
          It       -3.04       -3.09        0.04
          is       -2.45       -2.46        0.02
       silly       -6.08       -5.34       -0.75
           ,       -2.40       -2.39       -0.01
     kitschy       -9.86       -9.66       -0.20
          as       -2.73       -2.82        0.09
        hell       -6.11       -5.48       -0.63
           -       -4.05       -4.02       -0.03
         BUT       -7.83       -7.63       -0.20
           !       -3.31       -3.27       -0.04
  Definitely       -6.91       -7.41        0.50
         FUN       -9.58      -10.11        0.54
           !       -

**For Milestone 2 of the Project exercise you should perform error analysis on your baseline models and discuss your findings, including implications for how your solution could be improved**

## Ngram language modeling

__Ngrams are the sequences of words in a text__

In [43]:
text = 'This is the most famous movie by Aaron Sorkin'

In [44]:
unigrams = text.split()
print(unigrams)

['This', 'is', 'the', 'most', 'famous', 'movie', 'by', 'Aaron', 'Sorkin']


In [45]:
bigrams = [unigrams[i:i+2] for i in range(len(unigrams) - 1)]
print(bigrams)

[['This', 'is'], ['is', 'the'], ['the', 'most'], ['most', 'famous'], ['famous', 'movie'], ['movie', 'by'], ['by', 'Aaron'], ['Aaron', 'Sorkin']]


In [46]:
trigrams = [unigrams[i:i+3] for i in range(len(unigrams) - 2)]
print(trigrams)

[['This', 'is', 'the'], ['is', 'the', 'most'], ['the', 'most', 'famous'], ['most', 'famous', 'movie'], ['famous', 'movie', 'by'], ['movie', 'by', 'Aaron'], ['by', 'Aaron', 'Sorkin']]


Bag-of-ngrams are a natural extension of bag-of-words

The distribution of ngrams is also characteristic of a language

Ngram frequencies in a language can be used to approximate the __probability of any text__

This is known as __ngram language modeling__

#### Why model the probability of strings?

- autocomplete, autocorrect
- speech recognition
- machine translation
- augmentative and alternative communication (AAC)
- etc.

Consider the probability of an upcoming word

$$P(w_i | w_1, w_2, \ldots, w_{i-1})$$

An ngram language model __approximates__ this with the last $n-1$ words, i.e. by looking at a window of $n$ words only.

#### Trigram language model

$$P(w_i | w_{i-2}, w_{i-1})$$

This can be approximated based on counts of trigrams and bigrams in a corpus:

$$P(w_i | w_{i-2}, w_{i-1}) \sim \frac{c(w_{i-2}, w_{i-1}, w_{i})}{c(w_{i-2}, w_{i-1})}$$

(The larger the value of $n$, the higher the number of parameters, and even very large corpora become small)

In [47]:
trigrams = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
with open('data/shakespeare.txt') as f:
    for line in tqdm(f):
        words = tuple(['<s>'] + [w.lower() for w in nltk.word_tokenize(line)] + ['</s>'])
        for i in range(len(words) - 2):
            w1, w2, w3 = words[i:i+3]
            trigrams[w1][w2][w3] += 1


124225it [00:19, 6292.15it/s]


In [48]:
def get_next_word(words):
    assert len(words) >= 2
    w1, w2 = words[-2].lower(), words[-1].lower()
    words, freqs = zip(*trigrams[w1][w2].items())
    p = np.array(freqs) / sum(freqs)
    return np.random.choice(words, p=p)
    

In [49]:
get_next_word(('<s>', 'I'))

'kill'

In [50]:
def generate_sentence(first_word):
    text = ['<s>', first_word.lower()]
    while text[-1] != '</s>':
        text.append(get_next_word(text))
    return ' '.join(text[1:-1])

In [51]:
generate_sentence('She')

'she is not past a pint , as queen ,'

_"If one’s view of language is that it is a probability
distribution over strings of letter or sounds, one turns one’s back on the scientific
achievements of the ages and foreswears the opportunity that computers offer to carry
that enterprise forward."_ [(Kay 2006, p.430)](https://aclanthology.org/J05-4001.pdf)